In [6]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt



pd.set_option('display.max_columns', None)

pd.set_option("max_rows", None)

In [4]:
import pandas as pd
df=pd.read_csv("heart.csv")

In [3]:
def missing_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Missing Values'})
        mz_table['Attributes'] = df.nunique()
        mz_table['Data Type'] = df.dtypes
        
        
        
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
               )

        return mz_table

missing_values_table(df)

Your selected dataframe has 14 columns and 303 Rows.



,Zero Values,Missing Values,% of Total Missing Values,Attributes,Data Type
age,0,0,0.0,41,int64
sex,96,0,0.0,2,int64
cp,143,0,0.0,4,int64
trestbps,0,0,0.0,49,int64
chol,0,0,0.0,152,int64
fbs,258,0,0.0,2,int64
restecg,147,0,0.0,3,int64
thalach,0,0,0.0,91,int64
exang,204,0,0.0,2,int64
oldpeak,99,0,0.0,40,float64


In [9]:
#Lets chcek the value counts of each column of object datatype
for col in df.columns:
    if df[col].dtype=="int64":
        print(df[col].value_counts())
        print()

58    19
57    17
54    16
59    14
52    13
51    12
62    11
44    11
60    11
56    11
64    10
41    10
63     9
67     9
55     8
45     8
42     8
53     8
61     8
65     8
43     8
66     7
50     7
48     7
46     7
49     5
47     5
39     4
35     4
68     4
70     4
40     3
71     3
69     3
38     3
34     2
37     2
77     1
76     1
74     1
29     1
Name: age, dtype: int64

1    207
0     96
Name: sex, dtype: int64

0    143
2     87
1     50
3     23
Name: cp, dtype: int64

120    37
130    36
140    32
110    19
150    17
138    13
128    12
125    11
160    11
112     9
132     8
118     7
135     6
108     6
124     6
145     5
134     5
152     5
122     4
170     4
100     4
142     3
115     3
136     3
105     3
180     3
126     3
102     2
94      2
144     2
178     2
146     2
148     2
129     1
165     1
101     1
174     1
104     1
172     1
106     1
156     1
164     1
192     1
114     1
155     1
117     1
154     1
123     1
200     1
Name: trestbp

In [10]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [11]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [12]:
#Lets divide the dataset into input and output
df_x=df.drop(columns=["target"])
y=df[["target"]]

In [17]:
#Lets chcek the shape
df_x.shape
#Number column increased from 20 to 30 in input as a result of one hot encoding

(303, 13)

In [21]:
#since it is imbalanced dataset so we will focus on auc-roc score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
def max_aucroc_score(clf,df_x,y):
    max_aucroc_score=0
    for r_state in range(42,100):
        x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = r_state,test_size=0.20,stratify=y)
        x_train, y_train = SMOTE().fit_resample(x_train, y_train)
        clf.fit(x_train,y_train)
        y_pred = clf.predict(x_test)
        aucroc_scr=roc_auc_score(y_test,y_pred)
        print("auc roc score corresponding to ",r_state," is ",aucroc_scr)
        if aucroc_scr>max_aucroc_score:
            max_aucroc_score=aucroc_scr
            final_r_state=r_state
    print("max auc roc score corresponding to ",final_r_state," is ",max_aucroc_score)
    return final_r_state

In [22]:
#lets make a function which evaluates the model using cross_val_score
from sklearn.model_selection import cross_val_score
def model_evaluation(model,x,y):
    scores=cross_val_score(model,x,y,cv=5,scoring="roc_auc")
    print("Mean roc auc score for classifier: ",scores.mean())
    print("standard deviation in roc auc score for classifier: ",scores.std())
    print(scores)

In [23]:
#Lets chcek logistic regression
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")
lg_clf=LogisticRegression()
max_aucroc_score(lg_clf,df_x,y)

auc roc score corresponding to  42  is  0.7483766233766235
auc roc score corresponding to  43  is  0.8198051948051949
auc roc score corresponding to  44  is  0.8170995670995672
auc roc score corresponding to  45  is  0.7608225108225107
auc roc score corresponding to  46  is  0.8170995670995672
auc roc score corresponding to  47  is  0.841991341991342
auc roc score corresponding to  48  is  0.8474025974025974
auc roc score corresponding to  49  is  0.8555194805194806
auc roc score corresponding to  50  is  0.8831168831168832
auc roc score corresponding to  51  is  0.8116883116883118
auc roc score corresponding to  52  is  0.9009740259740261
auc roc score corresponding to  53  is  0.7911255411255411
auc roc score corresponding to  54  is  0.8982683982683983
auc roc score corresponding to  55  is  0.775974025974026
auc roc score corresponding to  56  is  0.7813852813852814
auc roc score corresponding to  57  is  0.83495670995671
auc roc score corresponding to  58  is  0.7732683982683983
a

52

In [24]:
#lets print the scores for logistic regression
print("LOGISTIC CLASSIFIER\n\n")
model_evaluation(lg_clf,df_x,y)

LOGISTIC CLASSIFIER


Mean roc auc score for classifier:  0.903896103896104
standard deviation in roc auc score for classifier:  0.03564172993766653
[0.9004329  0.9534632  0.87770563 0.93265993 0.85521886]


In [26]:
#Lets chcek the decision tree
from sklearn.tree import DecisionTreeClassifier
dc=DecisionTreeClassifier()
max_aucroc_score(dc,df_x,y)

auc roc score corresponding to  42  is  0.6547619047619047
auc roc score corresponding to  43  is  0.7619047619047619
auc roc score corresponding to  44  is  0.8100649350649352
auc roc score corresponding to  45  is  0.7402597402597402
auc roc score corresponding to  46  is  0.7207792207792209
auc roc score corresponding to  47  is  0.702922077922078
auc roc score corresponding to  48  is  0.8170995670995672
auc roc score corresponding to  49  is  0.7386363636363636
auc roc score corresponding to  50  is  0.8003246753246752
auc roc score corresponding to  51  is  0.7813852813852814
auc roc score corresponding to  52  is  0.8555194805194806
auc roc score corresponding to  53  is  0.7813852813852814
auc roc score corresponding to  54  is  0.7992424242424243
auc roc score corresponding to  55  is  0.7716450216450217
auc roc score corresponding to  56  is  0.6726190476190477
auc roc score corresponding to  57  is  0.7689393939393939
auc roc score corresponding to  58  is  0.697510822510822

83

In [27]:
#Lets print the scores of decision tree
print("Decision tree CLASSIFIER\n\n")
model_evaluation(dc,df_x,y)

Decision tree CLASSIFIER


Mean roc auc score for classifier:  0.7625901875901876
standard deviation in roc auc score for classifier:  0.05561574691570741
[0.74675325 0.870671   0.75108225 0.73063973 0.71380471]


In [28]:
#lets chcek random forest also
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameters={"n_estimators":[10,100,500]}
rf_clf=RandomForestClassifier()
clf = GridSearchCV(rf_clf, parameters, cv=10,scoring="roc_auc")
clf.fit(df_x,y)
clf.best_params_

{'n_estimators': 500}

In [30]:
rf_clf=RandomForestClassifier(n_estimators=500)
max_aucroc_score(rf_clf,df_x,y)

auc roc score corresponding to  42  is  0.8268398268398269
auc roc score corresponding to  43  is  0.8225108225108225
auc roc score corresponding to  44  is  0.7564935064935066
auc roc score corresponding to  45  is  0.7554112554112554
auc roc score corresponding to  46  is  0.7840909090909092
auc roc score corresponding to  47  is  0.7332251082251082
auc roc score corresponding to  48  is  0.8679653679653682
auc roc score corresponding to  49  is  0.8198051948051949
auc roc score corresponding to  50  is  0.83495670995671
auc roc score corresponding to  51  is  0.7965367965367965
auc roc score corresponding to  52  is  0.8858225108225108
auc roc score corresponding to  53  is  0.8116883116883118
auc roc score corresponding to  54  is  0.8322510822510822
auc roc score corresponding to  55  is  0.808982683982684
auc roc score corresponding to  56  is  0.7386363636363636
auc roc score corresponding to  57  is  0.8858225108225108
auc roc score corresponding to  58  is  0.808982683982684
a

52

In [31]:
#lets chcek KNN
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")
kc=KNeighborsClassifier()
neighbors={"n_neighbors":range(1,30)}
clf = GridSearchCV(kc, neighbors, cv=5,scoring="roc_auc")
clf.fit(df_x,y)
clf.best_params_

{'n_neighbors': 29}

In [32]:
kc=KNeighborsClassifier(n_neighbors=29)
max_aucroc_score(kc,df_x,y)

auc roc score corresponding to  42  is  0.6396103896103895
auc roc score corresponding to  43  is  0.6861471861471862
auc roc score corresponding to  44  is  0.6217532467532467
auc roc score corresponding to  45  is  0.6369047619047619
auc roc score corresponding to  46  is  0.579004329004329
auc roc score corresponding to  47  is  0.5930735930735931
auc roc score corresponding to  48  is  0.7619047619047619
auc roc score corresponding to  49  is  0.7564935064935066
auc roc score corresponding to  50  is  0.6931818181818181
auc roc score corresponding to  51  is  0.5021645021645021
auc roc score corresponding to  52  is  0.702922077922078
auc roc score corresponding to  53  is  0.619047619047619
auc roc score corresponding to  54  is  0.7110389610389609
auc roc score corresponding to  55  is  0.5335497835497836
auc roc score corresponding to  56  is  0.7056277056277056
auc roc score corresponding to  57  is  0.6574675324675324
auc roc score corresponding to  58  is  0.6645021645021645


78

In [33]:
#Lets chcek SVM Classifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc=SVC()
parameters={"kernel":["linear", "poly", "rbf"],"C":[0.001,0.01,0.1,1,10]}
clf = GridSearchCV(svc, parameters, cv=5,scoring="roc_auc")
clf.fit(df_x,y)
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [34]:
svc=SVC(C=1,kernel="linear")
max_aucroc_score(svc,df_x,y)

auc roc score corresponding to  42  is  0.7786796536796537
auc roc score corresponding to  43  is  0.801948051948052
auc roc score corresponding to  44  is  0.7716450216450217
auc roc score corresponding to  45  is  0.7786796536796537
auc roc score corresponding to  46  is  0.801948051948052
auc roc score corresponding to  47  is  0.7965367965367965
auc roc score corresponding to  48  is  0.8474025974025974
auc roc score corresponding to  49  is  0.8706709956709957
auc roc score corresponding to  50  is  0.8652597402597402
auc roc score corresponding to  51  is  0.8268398268398269
auc roc score corresponding to  52  is  0.8501082251082251
auc roc score corresponding to  53  is  0.7456709956709956
auc roc score corresponding to  54  is  0.8858225108225108
auc roc score corresponding to  55  is  0.7732683982683983
auc roc score corresponding to  56  is  0.7840909090909092
auc roc score corresponding to  57  is  0.8501082251082251
auc roc score corresponding to  58  is  0.775974025974026


94

In [35]:
#Cross val score for SVM classifier
print("SVM CLASSIFIER\n\n")
model_evaluation(svc,df_x,y)

SVM CLASSIFIER


Mean roc auc score for classifier:  0.9025172358505691
standard deviation in roc auc score for classifier:  0.03976796702414642
[0.9025974  0.95995671 0.87337662 0.92929293 0.84736251]


In [43]:
import lightgbm as lgb
lg = lgb.LGBMClassifier(silent=False)
param_dist = {"madf_x_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 5, scoring="roc_auc")
grid_search.fit(df_x,y)
grid_search.best_params_

[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Info] Number of positive: 165, number of negative: 138
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 303, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.544554 -> initscore=0.178692
[LightGBM] [Info] Start training from score 0.178692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

{'learning_rate': 0.01,
 'madf_x_depth': 25,
 'n_estimators': 200,
 'num_leaves': 300}

In [44]:
#LGBM also give provision to define which all columns should be treated as categorical
#So we will use above provision and see what is the result
params={'learning_rate': 0.01,'madf_x_depth': 25,'n_estimators': 200,'num_leaves': 300}
max_aucroc_score=0
for r_state in range(42,100):
    df_x_train, df_x_test, y_train, y_test = train_test_split(df_x, y,random_state = r_state,test_size=0.20,stratify=y)
    df_x_train, y_train = SMOTE().fit_resample(df_x_train, y_train)
    d_train = lgb.Dataset(df_x_train, label=y_train)
    lgb_model = lgb.train(params, d_train)
    y_pred = lgb_model.predict(df_x_test)
    roc_scr=roc_auc_score(y_test,y_pred)
    print("roc score corresponding to ",r_state," is ",roc_scr)
    if roc_scr>max_aucroc_score:
        max_aucroc_score=roc_scr
        final_r_state=r_state
print("max_aucroc_score corresponding to ",final_r_state," is ",max_aucroc_score)

[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 259
[LightGBM] [Info] Number of data points in the train set: 264, number of used features: 13
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
roc score corresponding to  43  is  0.9004329004329005
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Wa

roc score corresponding to  44  is  0.9058441558441559
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000388 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 256
[LightGBM] [Info] Number of data points in the train set: 264, number of used features: 13
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 261
[LightGBM] [Info] Number of data points in the train set: 264, number of used features: 13
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

roc score corresponding to  97  is  0.9112554112554113
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Unknown parameter: madf_x_depth
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 259
[LightGBM] [Info] Number of data points in the train set: 264, number of used features: 13
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# svc

In [53]:
#since from above we know maximum auc roc score we got for random state 48 so we will use it
x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = 94,test_size=0.20,stratify=y)

svc.fit(x_train,y_train)
y_pred=svc.predict(x_test)

In [54]:
#Lets print auc roc score and confusion matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix \n",confusion_matrix(y_test,y_pred))
print("auc roc score is : ",roc_auc_score(y_test,y_pred))
#so the best what we can get we got here and using above cross validation we verified that its not result of overfitting

Confusion matrix 
 [[24  4]
 [ 2 31]]
auc roc score is :  0.8982683982683983


# Logistic Regression

In [49]:
#since from above we know maximum auc roc score we got for random state 48 so we will use it
x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = 52,test_size=0.20,stratify=y)

lg_clf.fit(x_train,y_train)
y_pred=lg_clf.predict(x_test)

In [50]:
#Lets print auc roc score and confusion matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix \n",confusion_matrix(y_test,y_pred))
print("auc roc score is : ",roc_auc_score(y_test,y_pred))
#so the best what we can get we got here and using above cross validation we verified that its not result of overfitting

Confusion matrix 
 [[21  7]
 [ 3 30]]
auc roc score is :  0.8295454545454546


# decision Tree

In [51]:
#since from above we know maximum auc roc score we got for random state 48 so we will use it
x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = 83,test_size=0.20,stratify=y)

dc.fit(x_train,y_train)
y_pred=dc.predict(x_test)

In [52]:
#Lets print auc roc score and confusion matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix \n",confusion_matrix(y_test,y_pred))
print("auc roc score is : ",roc_auc_score(y_test,y_pred))
#so the best what we can get we got here and using above cross validation we verified that its not result of overfitting

Confusion matrix 
 [[24  4]
 [ 8 25]]
auc roc score is :  0.8073593073593074
